In [ ]:
import os
import pandas as pd
import zipfile

def transformar_fecha_hora(fecha_hora_str):
    if isinstance(fecha_hora_str, str) and len(fecha_hora_str) == 10:
        return f"{fecha_hora_str[:4]}-{fecha_hora_str[4:6]}-{fecha_hora_str[6:8]}-{fecha_hora_str[8:]}:00"
    return fecha_hora_str # Devuelve el valor original si no tiene el formato esperado

# Step 1: Read and process CSV files directly from .zip
def process_zip_files(zip_folder, output_file, csv_files_of_interest):
    periods = [2029, 2030, 2031, 2033, 2040, 2050]
    counter = 0
    for file in os.listdir(zip_folder):
        if file.endswith(".zip"):
            counter += 1
            consolidated_data_CMg = []
            consolidated_data_dispatch = []
            dispatch = []
            DispatchGen = []
            UnservedLoad = []
            zip_path = os.path.join(zip_folder, file)
            print(zip_path)
            with zipfile.ZipFile(zip_path, 'r') as archive:
                # List all files in the ZIP archive
                file_list = archive.namelist()
                # Process files in the expected folder structure
                files_of_interest = [f for f in file_list if f.endswith(tuple(csv_files_of_interest.keys()))]
                print(f"Archivos encontrados: {len(files_of_interest)}")
                for interest_file in files_of_interest:
                    with archive.open(interest_file) as f:

                        if 'marginal_cost' in interest_file:
                            df = pd.read_csv(f, header=None, names= ["load_zone", "period", "timepoint", "marginal_cost", "factor"], skiprows=1)

                            if len(df) == 0:
                                continue
                            
                            df['timepoint'] = pd.to_datetime(df['timepoint'], format='%Y-%m-%d-%H:%M')

                            df = df.sort_values(by=['load_zone', 'timepoint'])

                            # Calculate the difference in days between min and max timestamps
                            min_time = df['timepoint'].min()
                            max_time = df['timepoint'].max()
                            days_difference = (max_time - min_time).days

                            if days_difference > 7:
                                max_month = max_time.month
                                df = df[df['timepoint'].dt.month == max_month]

                            df_first_24_hours = df.groupby('load_zone').head(24)
                            # print(df_first_24_hours)
                            # Append the filtered data for this file to the consolidated data list
                            consolidated_data_CMg.append(df_first_24_hours)
                        
                        elif 'storage_dispatch' in interest_file or 'TES_dispatch' in interest_file:
                            df = pd.read_csv(f, delimiter=',')

                            if len(df) == 0:
                                continue

                            if "TES_dispatch" in interest_file:
                                rename_map = {"project": "generation_project", "ChargeTES_MWt": "ChargeMW", "DischargeTES_MWt": "DischargeMW", "TES_StateOfCharge_MWht": "StateOfCharge"}
                                df = df.rename(columns=rename_map)
                                df = df[["generation_project","timepoint","load_zone","ChargeMW","DischargeMW","StateOfCharge"]]

                            df['timepoint'] = pd.to_datetime(df['timepoint'], format='%Y-%m-%d-%H:%M')

                            df = df.sort_values(by=['generation_project', 'timepoint'])

                            # Calculate the difference in days between min and max timepoint
                            min_time = df['timepoint'].min()
                            max_time = df['timepoint'].max()
                            days_difference = (max_time - min_time).days

                            if days_difference > 7:
                                max_month = max_time.month
                                df = df[df['timepoint'].dt.month == max_month]
                            
                            df_first_24_hours = df.groupby('generation_project').head(24)
                            # print(df_first_24_hours)
                            # Append the filtered data for this file to the consolidated data list
                            consolidated_data_dispatch.append(df_first_24_hours)

                        elif 'dispatch.csv' in interest_file:
                            df = pd.read_csv(f, delimiter=',')

                            if len(df) == 0:
                                continue

                            df = df[["generation_project","timestamp","period","gen_load_zone","gen_tech","gen_energy_source","DispatchGen_MW","GenCapacity_MW"]]
                            
                            df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d-%H:%M')

                            df = df.sort_values(by=['generation_project', 'timestamp'])

                            # Calculate the difference in days between min and max timepoint
                            min_time = df['timestamp'].min()
                            max_time = df['timestamp'].max()
                            days_difference = (max_time - min_time).days

                            if days_difference > 7:
                                max_month = max_time.month
                                df = df[df['timestamp'].dt.month == max_month]

                            df_first_24_hours = df.groupby('generation_project').head(24)
                            # print(df_first_24_hours)
                            # Append the filtered data for this file to the consolidated data list
                            dispatch.append(df_first_24_hours)

                        elif 'DispatchGen' in interest_file:
                            df = pd.read_csv(f, delimiter=',')

                            if len(df) == 0:
                                continue

                            df = df[["GEN_TPS_1","GEN_TPS_2","DispatchGen"]]
                            df['GEN_TPS_2'] = df['GEN_TPS_2'].astype(str)
                            df['GEN_TPS_2'] = df['GEN_TPS_2'].apply(transformar_fecha_hora)
                            df['GEN_TPS_2'] = pd.to_datetime(df['GEN_TPS_2'], format='%Y-%m-%d-%H:%M')
                            df = df.sort_values(by=['GEN_TPS_1', 'GEN_TPS_2'])

                            # Calculate the difference in days between min and max timepoint
                            min_time = df['GEN_TPS_2'].min()
                            max_time = df['GEN_TPS_2'].max()
                            days_difference = (max_time - min_time).days

                            if days_difference > 7:
                                max_month = max_time.month
                                df = df[df['GEN_TPS_2'].dt.month == max_month]

                            df_first_24_hours = df.groupby('GEN_TPS_1').head(24)
                            # print(df_first_24_hours)
                            # Append the filtered data for this file to the consolidated data list
                            DispatchGen.append(df_first_24_hours)

                        # elif 'variable_capacity_factors.csv' in interest_file:
                        #     df = pd.read_csv(f, delimiter=',')

                        #     if len(df) == 0:
                        #         continue

                        #     df = df[["GENERATION_PROJECT","timepoint","gen_max_capacity_factor"]]
                        #     df['timepoint'] = df['timepoint'].astype(str)
                        #     df['timepoint'] = df['timepoint'].apply(transformar_fecha_hora)
                        #     df['timepoint'] = pd.to_datetime(df['timepoint'], format='%Y-%m-%d-%H:%M')
                        #     df = df.sort_values(by=['GENERATION_PROJECT', 'timepoint'])

                        #     # Calculate the difference in days between min and max timepoint
                        #     min_time = df['timepoint'].min()
                        #     max_time = df['timepoint'].max()
                        #     days_difference = (max_time - min_time).days

                        #     if days_difference > 7:
                        #         max_month = max_time.month
                        #         df = df[df['timepoint'].dt.month == max_month]

                        #     df_first_24_hours = df.groupby('GENERATION_PROJECT').head(24)
                        #     # print(df_first_24_hours)
                        #     # Append the filtered data for this file to the consolidated data list
                        #     variable_capacity_factors.append(df_first_24_hours)

                        elif 'UnservedLoad' in interest_file:
                            df = pd.read_csv(f, delimiter=',')

                            if len(df) == 0:
                                continue

                            df = df[["SetProduct_OrderedSet_1","SetProduct_OrderedSet_2","UnservedLoad"]]
                            df['SetProduct_OrderedSet_2'] = df['SetProduct_OrderedSet_2'].astype(str)
                            df['SetProduct_OrderedSet_2'] = df['SetProduct_OrderedSet_2'].apply(transformar_fecha_hora)
                            df['SetProduct_OrderedSet_2'] = pd.to_datetime(df['SetProduct_OrderedSet_2'], format='%Y-%m-%d-%H:%M')
                            df = df.sort_values(by=['SetProduct_OrderedSet_1', 'SetProduct_OrderedSet_2'])

                            # Calculate the difference in days between min and max timepoint
                            min_time = df['SetProduct_OrderedSet_2'].min()
                            max_time = df['SetProduct_OrderedSet_2'].max()
                            days_difference = (max_time - min_time).days

                            if days_difference > 7:
                                max_month = max_time.month
                                df = df[df['SetProduct_OrderedSet_2'].dt.month == max_month]

                            df_first_24_hours = df.groupby('SetProduct_OrderedSet_1').head(24)
                            # print(df_first_24_hours)
                            # Append the filtered data for this file to the consolidated data list
                            UnservedLoad.append(df_first_24_hours)

            # Combine all data into a single DataFrame
            if consolidated_data_CMg:
                result_df = pd.concat(consolidated_data_CMg, ignore_index=False)
                # result_df['timepoint'] = pd.to_datetime(result_df['timepoint'], format='%Y-%m-%d-%H:%M')
                result_df = result_df.sort_values(by=['load_zone', 'timepoint'])
                result_df.to_csv(os.path.join(zip_folder, f"consolidated_marginal_cost_{periods[counter-1]}.csv"), index=False)
                print(f"Consolidated data saved to consolidated_marginal_cost_{counter}.csv")
            else:
                print("No data to consolidate.")

            if consolidated_data_dispatch:
                result_df = pd.concat(consolidated_data_dispatch, ignore_index=False)
                # result_df['timepoint'] = pd.to_datetime(result_df['timepoint'], format='%Y-%m-%d-%H:%M')
                result_df = result_df.sort_values(by=['generation_project', 'timepoint'])
                result_df.to_csv(os.path.join(zip_folder, f"consolidated_dispatch_{periods[counter-1]}.csv"), index=False)
                print(f"Consolidated data saved to consolidated_dispatch_{counter}.csv")
            else:
                print("No data to consolidate.")

            if dispatch:
                result_df = pd.concat(dispatch, ignore_index=False)
                # result_df['timepoint'] = pd.to_datetime(result_df['timepoint'], format='%Y-%m-%d-%H:%M')
                result_df = result_df.sort_values(by=['generation_project', 'timestamp'])
                result_df.to_csv(os.path.join(zip_folder, f"dispatch_{periods[counter-1]}.csv"), index=False)
                print(f"Consolidated data saved to dispatch_{counter}.csv")
            else:
                print("No data to consolidate.")

            if DispatchGen:
                result_df = pd.concat(DispatchGen, ignore_index=False)
                # result_df['timepoint'] = pd.to_datetime(result_df['timepoint'], format='%Y-%m-%d-%H:%M')
                result_df = result_df.sort_values(by=['GEN_TPS_1', 'GEN_TPS_2'])
                result_df.to_csv(os.path.join(zip_folder, f"DispatchGen_{periods[counter-1]}.csv"), index=False)
                print(f"Consolidated data saved to DispatchGen_{counter}.csv")
            else:
                print("No data to consolidate.")
            
            # if variable_capacity_factors:
            #     result_df = pd.concat(variable_capacity_factors, ignore_index=False)
            #     # result_df['timepoint'] = pd.to_datetime(result_df['timepoint'], format='%Y-%m-%d-%H:%M')
            #     result_df = result_df.sort_values(by=['GENERATION_PROJECT', 'timepoint'])
            #     result_df.to_csv(os.path.join(zip_folder, f"variable_capacity_factors_{periods[counter-1]}.csv"), index=False)
            #     print(f"Consolidated data saved to variable_capacity_factors_{counter}.csv")
            # else:
            #     print("No data to consolidate.")

            if UnservedLoad:
                result_df = pd.concat(UnservedLoad, ignore_index=False)
                # result_df['timepoint'] = pd.to_datetime(result_df['timepoint'], format='%Y-%m-%d-%H:%M')
                result_df = result_df.sort_values(by=['SetProduct_OrderedSet_1', 'SetProduct_OrderedSet_2'])
                result_df.to_csv(os.path.join(zip_folder, f"UnservedLoad_{periods[counter-1]}.csv"), index=False)
                print(f"Consolidated data saved to UnservedLoad_{counter}.csv")
            else:
                print("No data to consolidate.")

# Paths (modify these as needed)
# zip_folder = os.path.join("d:/Github/switch_uai_scripts", "Modelos_SEN_finals/EEN 1 EEC 1 (Transicion Acelerada)")     # Folder containing .zip files
zip_folder = os.path.join(r"C:/Users/Ignac/Trabajo_Centra/Catedra-LDES/CII-Centra-EDF/Operacion", r"escenarios/CaseBase1/TA/")
print(zip_folder)
# zip_folder = os.path.join("d:/Github/switch_uai_scripts", "Modelos_SEN_finals/EEN -1 EEC -1 (Recuperacion Lenta)")
csv_files_of_interest = {
    "outputs_dispatch/marginal_cost.csv": {
        "columns": ["load_zone", "period", "timepoint", "marginal_cost", "factor"],
        "skiprows": 1
    },
    "outputs_dispatch/storage_dispatch.csv": {
        "columns": ["generation_project","timepoint","load_zone","ChargeMW","DischargeMW","StateOfCharge"],
        "skiprows": None
    },
    "outputs_dispatch/TES_dispatch.txt": {
        "columns": ["generation_project","timepoint","load_zone","ChargeMW","DischargeMW","StateOfCharge"],
        "skiprows": None
    },
    "outputs_dispatch/dispatch.csv": {
        "columns": ["generation_project","timestamp","period","gen_load_zone","gen_tech","gen_energy_source","DispatchGen_MW","GenCapacity_MW"],
        "skiprows": None
    },
    "outputs_dispatch/DispatchGen.csv": {
        "columns": ["GEN_TPS_1","GEN_TPS_2","DispatchGen"],
        "skiprows": None
    },
    "outputs_dispatch/UnservedLoad.csv": {
        "columns": ["SetProduct_OrderedSet_1","SetProduct_OrderedSet_2","UnservedLoad"],
        "skiprows": None
    },
    # "inputs_dispacth/variable_capacity_factors.csv": {
    #     "columns": ["GENERATION_PROJECT","timepoint","gen_max_capacity_factor"],
    #     "skiprows": None
    # },
}
# csv_files_of_interest = ["outputs_dispatch/marginal_cost.csv"]#, "outputs_dispatch/storage_dispatch.csv"]  # CSV files to process
os.listdir(zip_folder)
output_file = "consolidated_data.csv"  # Output file for consolidated data
# Run the process
process_zip_files(zip_folder, output_file, csv_files_of_interest)


In [ ]:
df_test = pd.read_csv(os.path.join("D:\Github\switch_uai_scripts\Modelos_SEN_finals\EEN 0 EEC 0 (Rumbo CNprueba)\outputs", "TES_dispatch.txt"))
len(df_test)